# Compute handcrafted and convolutional features for CC359 dataset
One of the most important task in a machine learning algorithm is the feature extraction. In traditional methods, these features describes a-priori information about the data, and are mainly set by the user (feature engeneering - handcrafted features). Nowadays, using deep learning methods, the most discriminating features are automatically detected by the network (convolutional features).

Even though convolutional features may lead to better results, they are more complex to describe than the handcrafted features, such as intensity or texture. Thus, specially when dealing with medical imaging, a combination of these features may be desired. (Further information about this features combination: acess ([link](feats-CC-hand-conv.ipynb))

In [29]:
import sys
import os
import numpy as np
import nibabel as nib
from skimage import exposure, img_as_float, feature
from scipy import stats
from keras.applications.vgg16 import VGG16
import pywt
from skimage.transform import resize
from keras.applications.vgg16 import preprocess_input
from skimage.morphology import dilation,ball,erosion

MY_UTILS_PATH = "../Modules/"
if not MY_UTILS_PATH in sys.path:
    sys.path.append(MY_UTILS_PATH)
import ipt_utils

model = VGG16(weights='imagenet', include_top=False)

def feature_extraction(f,fs):
    att = []
    att_type = []
    for i in fs:
        if i == 'fs1':   # intensity based attributes (histogram; statistics)
            hist = np.bincount(np.ravel(f[f>0]))
            att.append([stats.entropy(hist),f[f>0].mean(),f[f>0].std(),stats.skew(hist),stats.kurtosis(hist),np.percentile(f[f>0], 10),np.percentile(f[f>0], 50),np.percentile(f[f>0], 90)])
            att_type.append(np.ones((8)))
        elif i == 'fs2': # gradient based attributes (gray level, morphological, hessian)
            aux = np.gradient(f)
            grad = np.sqrt(aux[0]**2+aux[1]**2+aux[2]**2)
            mgrad = dilation(f,ball(1))-erosion(f,ball(1))
            att.append([grad[grad>0].mean(), grad[grad>0].std(), stats.skew(np.ravel(grad[grad>0])), stats.kurtosis(np.ravel(grad[grad>0])), 1.0*np.count_nonzero(grad)/np.size(grad),mgrad[mgrad>0].mean(), mgrad[mgrad>0].std(), stats.skew(np.ravel(mgrad[mgrad>0])),stats.kurtosis(np.ravel(mgrad[mgrad>0])), 1.0*np.count_nonzero(mgrad)/np.size(mgrad)])
            att_type.append(2*np.ones((10)))
        elif i == 'fs3': # strutural texture (lbp)
            lbp = ipt_utils.lbp3D(f)
            bins,_ = np.histogram(lbp, bins=10,density = True)
            att.append(bins)
            att_type.append(3*np.ones((10)))
        elif i == 'fs4': # frequence domain (ft, wavelet)
            coeffs = pywt.dwtn(f, 'haar')
            att.append([coeffs['dad'].mean(),coeffs['aad'].mean(),coeffs['aaa'].mean(),coeffs['daa'].mean(),coeffs['add'].mean(),coeffs['ada'].mean(),coeffs['dda'].mean(),coeffs['ddd'].mean()])
            att_type.append(4*np.ones((8)))
        elif i == 'fs5': # convolutional networks
            if len(f.shape)==3:
                for k in xrange(3):
                    if k==0:
                        faux = f[f.shape[0]/2]
                    elif k==1:
                        faux = f[:,f.shape[1]/2]
                    elif k==2:
                        faux = f[:,:,f.shape[2]/2]
                    img = np.zeros((224,224,3))
                    img[:,:,0] = img[:,:,1] = img[:,:,2] = ipt_utils.intensity_normalization(resize(faux, (224,224), mode='reflect'))
                    x = np.expand_dims(img, axis=0)
                    x = preprocess_input(x)
                    att.append(model.predict(x).ravel())
                    att_type.append(5*np.ones((75264)))
    return np.concatenate(att),np.concatenate(att_type)


## Reading images and computing all features from each image

In [30]:
## Saving in a file the corresponding characteristics (vendor, magnetic field, patient age, patient gender) and features

data = np.zeros((359,75304))
root = '../Data/CC359/'
nimage = 0
for path, subdirs, files in os.walk(root):
    for name in files :
        if '.nii' in name:
            img = nib.load(root+name)
            img_data = ipt_utils.intensity_normalization(img.get_data(),NEW_MAX = 256).astype(np.uint8)
            feats,_ = np.array(feature_extraction(img_data,['fs1','fs2','fs3','fs4','fs5']))
            if name.split('_')[1].lower() == 'ge':
                data[nimage][0] = 10
            elif name.split('_')[1].lower() == 'philips':
                data[nimage][0] = 11
            elif name.split('_')[1].lower() == 'siemens':
                data[nimage][0] = 12    
            data[nimage][1] = int(name.split('_')[2])
            data[nimage][2] = int(name.split('_')[3])
            if name.split('_')[-1].split('.')[0].lower() == 'f':
                data[nimage][3] = 10
            elif name.split('_')[-1].split('.')[0].lower() == 'm':
                data[nimage][3] = 11
            data[nimage,4:] = feats
            nimage+=1
            
            print '/file/',name.split('_')[1],name.split('_')[2],name.split('_')[3],name.split('_')[-1].split('.')[0],feats[0],';/save/',data[nimage-1,:5]
            
np.save(os.path.join('../Data/'+"feats_cc359"),data)            

/file/ philips 15 55 M 5.11600527056 ;/save/ [ 11.          15.          55.          11.           5.11600527]
/file/ philips 15 56 M 4.38476085115 ;/save/ [ 11.          15.          56.          11.           4.38476085]
/file/ philips 15 63 F 4.52892385574 ;/save/ [ 11.          15.          63.          10.           4.52892386]
/file/ philips 15 67 M 4.71551215087 ;/save/ [ 11.          15.          67.          11.           4.71551215]
/file/ philips 15 62 M 4.57782883338 ;/save/ [ 11.          15.          62.          11.           4.57782883]
/file/ philips 15 63 F 4.4148757279 ;/save/ [ 11.          15.          63.          10.           4.41487573]
/file/ philips 15 62 M 4.60926663745 ;/save/ [ 11.          15.          62.          11.           4.60926664]
/file/ philips 15 60 F 4.75445275014 ;/save/ [ 11.          15.          60.          10.           4.75445275]
/file/ philips 15 69 M 4.70290061174 ;/save/ [ 11.          15.          69.          11.           4.702

/file/ philips 3 41 M 5.07954129705 ;/save/ [ 11.          3.         41.         11.          5.0795413]
/file/ philips 3 44 F 5.25839552417 ;/save/ [ 11.           3.          44.          10.           5.25839552]
/file/ philips 3 43 F 5.16723808824 ;/save/ [ 11.           3.          43.          10.           5.16723809]
/file/ philips 3 42 M 5.05107187667 ;/save/ [ 11.           3.          42.          11.           5.05107188]
/file/ philips 3 47 F 5.15375760653 ;/save/ [ 11.           3.          47.          10.           5.15375761]
/file/ philips 3 55 F 5.05921989605 ;/save/ [ 11.          3.         55.         10.          5.0592199]
/file/ philips 3 67 F 4.99908265411 ;/save/ [ 11.           3.          67.          10.           4.99908265]
/file/ philips 3 64 F 5.03934134966 ;/save/ [ 11.           3.          64.          10.           5.03934135]
/file/ philips 3 66 M 4.97223316312 ;/save/ [ 11.           3.          66.          11.           4.97223316]
/file/ phil

/file/ siemens 15 53 M 4.43648334022 ;/save/ [ 12.          15.          53.          11.           4.43648334]
/file/ siemens 15 47 M 4.29272514405 ;/save/ [ 12.          15.          47.          11.           4.29272514]
/file/ siemens 15 71 M 3.99540208436 ;/save/ [ 12.          15.          71.          11.           3.99540208]
/file/ siemens 15 51 M 4.36047755336 ;/save/ [ 12.          15.          51.          11.           4.36047755]
/file/ siemens 15 52 F 3.81951233432 ;/save/ [ 12.          15.          52.          10.           3.81951233]
/file/ siemens 15 55 M 4.22625218434 ;/save/ [ 12.          15.          55.          11.           4.22625218]
/file/ siemens 15 57 F 3.73999263127 ;/save/ [ 12.          15.          57.          10.           3.73999263]
/file/ siemens 15 55 M 4.07121120031 ;/save/ [ 12.         15.         55.         11.          4.0712112]
/file/ siemens 15 54 F 4.04561091386 ;/save/ [ 12.          15.          54.          10.           4.0456109

/file/ siemens 3 56 M 3.51786111384 ;/save/ [ 12.           3.          56.          11.           3.51786111]
/file/ siemens 3 62 M 3.70019406722 ;/save/ [ 12.           3.          62.          11.           3.70019407]
/file/ siemens 3 47 F 3.50007242128 ;/save/ [ 12.           3.          47.          10.           3.50007242]
/file/ siemens 3 64 F 3.60006821324 ;/save/ [ 12.           3.          64.          10.           3.60006821]
/file/ siemens 3 58 M 3.50408762801 ;/save/ [ 12.           3.          58.          11.           3.50408763]
/file/ siemens 3 66 M 3.55124465302 ;/save/ [ 12.           3.          66.          11.           3.55124465]
/file/ siemens 3 55 F 3.61350184615 ;/save/ [ 12.           3.          55.          10.           3.61350185]
/file/ siemens 3 60 M 3.7562181809 ;/save/ [ 12.           3.          60.          11.           3.75621818]
/file/ siemens 3 56 M 3.62495819142 ;/save/ [ 12.           3.          56.          11.           3.62495819]
/f

/file/ ge 3 58 M 4.17488943384 ;/save/ [ 10.           3.          58.          11.           4.17488943]
/file/ ge 3 53 F 4.06866072726 ;/save/ [ 10.           3.          53.          10.           4.06866073]
/file/ ge 3 56 F 4.31854402785 ;/save/ [ 10.           3.          56.          10.           4.31854403]
/file/ ge 3 53 M 4.1304756182 ;/save/ [ 10.           3.          53.          11.           4.13047562]
/file/ ge 3 57 F 4.03355644579 ;/save/ [ 10.           3.          57.          10.           4.03355645]
/file/ ge 3 59 M 4.36085079231 ;/save/ [ 10.           3.          59.          11.           4.36085079]
/file/ ge 3 52 M 4.17035893673 ;/save/ [ 10.           3.          52.          11.           4.17035894]
/file/ ge 3 57 M 4.28780781899 ;/save/ [ 10.           3.          57.          11.           4.28780782]
/file/ ge 3 52 F 4.00939602433 ;/save/ [ 10.           3.          52.          10.           4.00939602]
/file/ ge 3 42 M 4.60389208275 ;/save/ [ 10.   

# Verify if file was correctly saved

In [31]:
dataR = np.load(os.path.join('../Data/'+"feats_cc359.npy"))
print dataR.shape

(359, 75304)


# References
- Keras Applications on deep learning models alongside pre-trained weights: https://keras.io/applications/
- texture analysis with skimage: http://scikit-image.org/docs/0.7.0/api/skimage.feature.texture.html
